# Upload Data from Dave's Redistricting

Due to discrepancies in 538's data and untraceable sourcing, I have pulled together public data from Dave's Redistricting App on 3.20.2022.

At the point of initial pull, Missouri, Louisiana, New Hampshire, and Florida still had incomplete maps. 

Also, Kentucky and West Virginia, for some reason, did not yet have data for the 2020 Presidential election yet. Ohio's new map was also not included. For these states, we'll still have to use 538's less traceable data.

In [1]:
import requests
import pandas as pd
import os
import glob
import numpy as np
import math
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
#load the files into a single dataframe
path = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/daves_pvis"
all_files = sorted(glob.glob(path + "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0).reset_index()
    df["ST"] = filename[-6:-4]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=False)

In [3]:
#cut the frame and fix the columns
df = frame[["ST","index","Devation","Dem"]]
df.columns = ["ST","district","dem_pct","gop_pct"]
#remove nonumeric rows
possible_numbers = list([str(i) for i in range(53)])
df = df[df["district"].isin(possible_numbers)]
df.shape

(362, 4)

Without Florida, Ohio, Missouri, Kentucky, Louisiana, New Hampshire having valid data, and the 6 single district states not being in the set, 362 is the exact number we should have

# Clean the Dave's data into our standardized format

In [4]:
#Input standard text values
df["year"] = "2022"
df["congress"] = "118"
df["ST#"] = df["ST"] + df["district"]
#compute metric
df["raw_metric"] = (.50 + df["gop_pct"] - .4831) * 100
df["round_metric"] = df["raw_metric"].apply(np.around)
df["metric"] = round(df["raw_metric"]/100,2)
#compute PVI
df["lean"] = np.where(df["metric"] > .5, "R", "D")
df['PVI'] = df['lean'] + "+" + ((((df['round_metric']-50)).abs())).fillna(1000).astype(int).astype(str)
df['PVI'] = df['PVI'].replace("D+0","EVEN")
#df['PVI'] = np.where(df['PVI'].str[-1]=="+","EVEN",df['PVI'])
#df["metric"] = df['PVI'].str.split("+").str[1]
#df['metric'] = np.where(df['PVI']=="EVEN","50",df['metric'])
#df["metric"] = df["metric"].astype(int)
#df["metric"] = np.where(df["lean"]=="D", (df["metric"]/100))
#checker = df[["raw_metric","round_metric","metric","PVI"]]
#checker

In [5]:
#df[abs(df["round_metric"] - df["raw_metric"]) > .4]
df

,ST,district,dem_pct,gop_pct,year,congress,ST#,raw_metric,round_metric,metric,lean,PVI
1,AL,1,0.3535,0.6465,2022,118,AL1,66.34,66.0,0.66,R,R+16
2,AL,2,0.3423,0.6577,2022,118,AL2,67.46,67.0,0.67,R,R+17
3,AL,3,0.3248,0.6752,2022,118,AL3,69.21,69.0,0.69,R,R+19
4,AL,4,0.1870,0.8130,2022,118,AL4,82.99,83.0,0.83,R,R+33
5,AL,5,0.3457,0.6543,2022,118,AL5,67.12,67.0,0.67,R,R+17
6,AL,6,0.3253,0.6747,2022,118,AL6,69.16,69.0,0.69,R,R+19
7,AL,7,0.6596,0.3404,2022,118,AL7,35.73,36.0,0.36,D,D+14
1,AR,1,0.3025,0.6975,2022,118,AR1,71.44,71.0,0.71,R,R+21
2,AR,2,0.4267,0.5733,2022,118,AR2,59.02,59.0,0.59,R,R+9
3,AR,3,0.3638,0.6362,2022,118,AR3,65.31,65.0,0.65,R,R+15


In [6]:
#create a clean version of the data to match other outputs
pvi_118 = df[["year","congress","ST","ST#","PVI","metric"]]

# Load in 538 Data to fill in gaps

In [7]:
#load in filepaths for 538's data
path_538 = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/supplimental/data_118_538.csv"
path_pre = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/supplimental/previews_118.csv"

In [8]:
#grab states with maps not available in Dave's
no_data_states = ["KY","WV","OH","VT", "DE", "WY", "ND", "SD", "AK"]
load_538 = pd.read_csv(path_538)
no_data_rows = load_538[load_538["ST"].isin(no_data_states)]
#pull in maps not yet passed
pre_rows = pd.read_csv(path_pre)

In [9]:
pvi_118 = pd.concat([pvi_118,no_data_rows,pre_rows]).sort_values("ST")
pvi_118.shape

(435, 6)

Now we have something to represent all 435 Districts, we can export a dataset with the best possible estimates of what the 2022 map will look like

In [10]:
pvi_118.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts/data_118.csv",index=False)

In [11]:
pvi_118[pvi_118["metric"].isna()]

,year,congress,ST,ST#,PVI,metric
